In [41]:
r=requests.get('https://github.com/mlukan/GDA3B2021/raw/main/ProjectFaceMask/annotdf.csv')
from io import StringIO
#StringIO("""col1;col2;col3
andf=pd.read_csv(StringIO(r.text),sep=';')

In [42]:
andf

,Unnamed: 0,xmin,ymin,xmax,ymax,label,file,width,height,depth
0,0,76,236,149,349,without_mask,maksssksksss530.png,301,400,3
1,1,241,60,271,97,with_mask,maksssksksss367.png,400,267,3
2,2,89,169,215,309,without_mask,maksssksksss442.png,301,400,3
3,3,156,80,222,155,with_mask,maksssksksss242.png,400,300,3
4,4,75,75,158,161,with_mask,maksssksksss215.png,400,400,3
...,...,...,...,...,...,...,...,...,...,...
4067,4067,235,43,272,87,with_mask,maksssksksss116.png,400,225,3
4068,4068,304,68,336,102,with_mask,maksssksksss116.png,400,225,3
4069,4069,379,61,399,96,with_mask,maksssksksss116.png,400,225,3
4070,4070,89,210,180,303,with_mask,maksssksksss339.png,400,400,3


In [48]:
import xmltodict
import os
def parse_file(path,file=None):
    '''
    parses xml file into list of dictionaries
    '''
    path=path+file
    
    def parse_object(obj,metadata):
        #Parse annotation['object'] into a dictionary including parent metadata
        objdict={}
        objdict = obj['bndbox']
        #print(objdict)
        objdict['label']=obj['name']
        outdict = {**objdict,**metadata}
        return outdict
    #print(path)
    with open(path,'r') as f:
        #Read xml file and parse it into an ordered dictionary
        parsed=xmltodict.parse(f.read())
        #Select just the key 'annotation' for further processing
        annot=parsed['annotation']
        
        parselist=[]
        metadata={}
        metadata['file']=annot['filename']
        size= annot['size']
        #Collect  image metadata
        metadata={**metadata,**size}
        #print('Metadata',metadata)
        #Extract data about labeled objects
        if type(annot['object']) is list:
            for obj in annot['object']:
                parselist.append(parse_object(obj,metadata))
        else:
            parselist.append(parse_object(annot['object'],metadata))
        return parselist
#parse_file(os.getcwd()+'/','test.xml')


In [49]:
#Create a list of dictionaries of fall xml files in a specified directory
import os
import pandas as pd
datalist=[]
for file in os.listdir('annotations'):
    if 'xml' in file:
        objlist=parse_file('./annotations/',file)
        datalist.extend(objlist)
annotdf=pd.DataFrame(datalist)


In [50]:
annotdf

,xmin,ymin,xmax,ymax,label,file,width,height,depth
0,76,236,149,349,without_mask,maksssksksss530.png,301,400,3
1,241,60,271,97,with_mask,maksssksksss367.png,400,267,3
2,89,169,215,309,without_mask,maksssksksss442.png,301,400,3
3,156,80,222,155,with_mask,maksssksksss242.png,400,300,3
4,75,75,158,161,with_mask,maksssksksss215.png,400,400,3
...,...,...,...,...,...,...,...,...,...
4067,235,43,272,87,with_mask,maksssksksss116.png,400,225,3
4068,304,68,336,102,with_mask,maksssksksss116.png,400,225,3
4069,379,61,399,96,with_mask,maksssksksss116.png,400,225,3
4070,89,210,180,303,with_mask,maksssksksss339.png,400,400,3


In [43]:
annotdf.groupby(by='label').count().sort_values(by='label',ascending=False)

,xmin,ymin,xmax,ymax,file,width,height,depth
label,,,,,,,,
without_mask,717,717,717,717,717,717,717,717
with_mask,3232,3232,3232,3232,3232,3232,3232,3232
mask_weared_incorrect,123,123,123,123,123,123,123,123


In [9]:
annotdf[annotdf['file']=='maksssksksss603.png']

,xmin,ymin,xmax,ymax,label,file,width,height,depth
1325,49,11,55,17,with_mask,maksssksksss603.png,400,278,3
1326,32,52,40,60,with_mask,maksssksksss603.png,400,278,3
1327,52,76,61,86,without_mask,maksssksksss603.png,400,278,3
1328,47,98,58,109,with_mask,maksssksksss603.png,400,278,3
1329,68,100,75,111,with_mask,maksssksksss603.png,400,278,3
...,...,...,...,...,...,...,...,...,...
1435,356,18,362,24,with_mask,maksssksksss603.png,400,278,3
1436,346,9,351,16,with_mask,maksssksksss603.png,400,278,3
1437,370,13,376,18,with_mask,maksssksksss603.png,400,278,3
1438,382,6,386,12,without_mask,maksssksksss603.png,400,278,3


In [28]:
import PIL

